In [1]:
import tkinter as tk
import cv2
from PIL import Image
from PIL import ImageTk
from tkinter import messagebox

class AppWindow(tk.Frame):                #frame 상속
    def __init__(self, master=None, size=None):
        super().__init__(master)
        self.master = master              #Tk 객체
        self.master.geometry(size)        #창크기,위치 조절
        self.master.resizable(True, True)
        self.pack()                       #opencv frame # 자기 자신이 프레임이니까 붙임
        self.sub_fr = None                #frame
        self.src = None                   #tk의 label에 출력할 영상
        self.label = None                 #tk의 영상을 출력할 레이블
        self.image = None                 #사진저장에 사용할 이미지
        self.camera = CameraService(self)
        self.create_widgets()              #위젯 초기화

    def make_img(self, path):#path 경로의 이미지를 레이블에 출력
        src = cv2.imread(path)
        src = cv2.resize(src, (640, 480))
        img = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)#넘파이 배열을 이미지로 변환
        self.src = ImageTk.PhotoImage(image=img)#tkinter에서 인식할 수 있는 이미지로 생성

    def create_widgets(self):#프레임에 위젯 추가
        self.make_img('img/app_intro.jpg')
        self.label = tk.Label(self, image=self.src)
        self.label.pack()
        self.sub_fr = tk.Frame(self.master)#frame
        self.sub_fr.pack()
        
        self.btn1 = tk.Button(self.sub_fr, width=10, font=60, text='사진찍기')
        self.btn2 = tk.Button(self.sub_fr, width=10, font=60, text='사진보기')
        self.btn3 = tk.Button(self.sub_fr, width=10, font=60, text='preview')
        self.btn4 = tk.Button(self.sub_fr, width=10, font=60, text='snow preview')
        self.ent = tk.Entry(self.sub_fr, width=60)
        self.btn5 = tk.Button(self.sub_fr, width=10, font=60, text='사진저장')
        self.btn6 = tk.Button(self.sub_fr, width=10, font=60, text='동영상촬영')
        self.btn7 = tk.Button(self.sub_fr, width=10, font=60, text='동영상보기')

        self.ent.grid(row=0, column=0, columnspan=5)
        self.btn1.grid(row=1, column=0)
        self.btn2.grid(row=1, column=1)
        self.btn3.grid(row=1, column=2)
        self.btn4.grid(row=1, column=3)
        self.btn5.grid(row=1, column=4)
        self.btn6.grid(row=2, column=0)
        self.btn7.grid(row=2, column=1)
        
        self.btn1['command'] = self.btn1_clicked
        self.btn3['command'] = self.btn3_clicked
        self.btn5['command'] = self.btn5_clicked
        self.btn4['command'] = self.btn4_clicked
        self.btn6['command'] = self.btn6_clicked
        self.btn7['command'] = self.btn7_clicked
        
    def btn6_clicked(self):
        print('동영상촬영')
        fileName = self.ent.get()
        if fileName=='' or fileName is None:
            messagebox.showinfo(title='error', message='파일명 입력하라')
            return
        self.camera.make_video(fileName)
        
    def btn7_clicked(self):
        print('동영상보기')  
        fileName = self.ent.get()
        if fileName=='' or fileName is None:
            messagebox.showinfo(title='error', message='파일명 입력하라')
            return
        self.camera.view_video(fileName)
    
    def btn1_clicked(self):
        print('preview 종료')
        self.camera.stop()
        
    def btn3_clicked(self):
        print('preview 시작')
        self.camera.preview()
        
    def btn4_clicked(self):
        print('preview_snow 시작')
        self.camera.preview_snow()
    
    def btn5_clicked(self):
        print('사진 저장')
        fileName = self.ent.get()
        if fileName=='' or fileName is None:
            messagebox.showinfo(title='error', message='파일명 입력하라')
            return
        cv2.imwrite('pic/'+fileName+'.jpg',self.image)
        
    def change_img(self, res):
        self.image = res
        res = cv2.resize(res, (640, 400))
        img = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        self.src = ImageTk.PhotoImage(image=img)
        self.label['image'] = self.src
        

In [4]:
import threading
import os

class CameraService:
    def __init__(self, app):
        self.app = app       #AppWindow 객체
        self.flag = False   #쓰레드 종료 조건
        
    def preview_th(self, stop):#쓰레드 종료 조건. stop이 True일동안 실행
        cam = cv2.VideoCapture(0)
        cam.set(3, 640)
        cam.set(4, 400)
        
        while stop():
            ret, fr = cam.read()
            if ret:
                app.change_img(fr)
            cv2.waitKey(100)
        cam.release()
    
    def preview(self):
        self.flag = True
        #쓰레드 생성
        th = threading.Thread(target=self.preview_th, args=(lambda:self.flag,))
        #쓰레드 실행
        th.start()
        
    def stop(self):  #preview_th()종료 함수
        self.flag = False
    
    def preview_snow_th(self, stop):#쓰레드 종료 조건. stop이 True일동안 실행
        cam = cv2.VideoCapture(0)
        cam.set(3, 640)
        cam.set(4, 400)
        
        #왕관 처리
        crown_img = cv2.imread('img/crown1.jpg')             
        c_h, c_w, _ = crown_img.shape    
        gray = cv2.cvtColor(crown_img, cv2.COLOR_BGR2GRAY)      
        _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        _, mask_inv = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
         #왕관 이미지에서 배경 빼고 왕관만 오려냄
        c_fg = cv2.bitwise_and(crown_img, crown_img, mask=mask_inv)
        
        #안경 처리
        glasses_img = cv2.imread('img/glasses.png')
        ggray = cv2.cvtColor(glasses_img, cv2.COLOR_BGR2GRAY)
        ggray[ggray[:]==0]=255   
        ggray[ggray[:]!=255]=0   #mask
        ggray_inv = cv2.bitwise_not(ggray)   #mask_inv
        
        #얼굴 분류기 생성
        face_classifier = cv2.CascadeClassifier('haar/haarcascade_frontalface_alt2.xml')
        while stop():
            ret, fr = cam.read()
            if ret:
                #영상 fr에서 얼굴 탐지
                faces = face_classifier.detectMultiScale(fr)

                for (x1, y1, w1, h1) in faces:
                    leftTop=[x1+w1//2-c_w//2, y1-c_h]
                    roi = fr[leftTop[1]:leftTop[1]+c_h, leftTop[0]:leftTop[0]+c_w]
                                     
                    #사람 얼굴의 왕관 들어갈 자리에서 왕관 모양을 뺀 나머지 부분 추출
                    roi_bg = cv2.bitwise_and(roi, roi, mask=mask) 
                    face_crown = cv2.add(roi_bg, c_fg)
                    roi[:] = face_crown[:]
                    
                    face_roi = fr[y1:y1+h1, x1:x1+w1]#검출된 얼굴 영역
                    gg = cv2.resize(glasses_img, (w1, h1))#칼라 안경 이미지 크기 얼굴 크기로 조절
                    gg_mask = cv2.resize(ggray, (w1, h1))
                    gg_mask_inv = cv2.resize(ggray_inv, (w1, h1))
                    gg_fg = cv2.bitwise_and(gg, gg, mask=gg_mask_inv)
                    gg_bg = cv2.bitwise_and(face_roi, face_roi, mask=gg_mask) 
                    ggl = cv2.add(gg_bg, gg_fg)
                    face_roi[:] = ggl[:]
                app.change_img(fr)
            cv2.waitKey(100)
        cam.release()
       
    def preview_snow(self):
        self.flag = True
        #쓰레드 생성
        th = threading.Thread(target=self.preview_snow_th, args=(lambda:self.flag,))
        #쓰레드 실행
        th.start()
        
    def make_video_th(self, stop, fname):
        cam = cv2.VideoCapture(0)
        codec = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter('pic/video/'+fname+'.mp4', codec, 25.0, (640, 400), True)
        while stop():
            ret, fr = cam.read()
            if ret:
                fr = cv2.resize(fr, (640, 400))
                writer.write(fr)
                app.change_img(fr)
            else:
                break
            
        cam.release()
    
    def make_video(self, fname):
        self.flag = True
        #쓰레드 생성
        th = threading.Thread(target=self.make_video_th, args=(lambda:self.flag, fname))
        #쓰레드 실행
        th.start()
    
    def view_video_th(self, stop, fname):
        path = 'pic/video/'+fname+'.mp4'
        if not os.path.isfile(path):
            print('파일 없음')
            return
        cam = cv2.VideoCapture(path)
        while stop():
            ret, fr = cam.read()
            if ret:
                app.change_img(fr)
            else:
                break
            cv2.waitKey(10)
        cam.release()
    
    def view_video(self, fname):
        self.flag = True
        #쓰레드 생성
        th = threading.Thread(target=self.view_video_th, args=(lambda:self.flag, fname))
        #쓰레드 실행
        th.start()

In [5]:
root = tk.Tk()
app = AppWindow(root, '700x700+100+100')
app.mainloop()#ui thread

동영상보기


In [38]:
crown_img = cv2.imread('img/crown1.jpg')
gray = cv2.cvtColor(crown_img, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
_, mask_inv = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)


c_fg = cv2.bitwise_and(crown_img, crown_img, mask=mask_inv)#왕관 이미지에서 배경 빼고 왕관만 오려냄
roi_bg = cv2.bitwise_and(roi, roi, mask=mask) #사람 얼굴의 왕관 들어갈 자리에서 왕관 모양을 뺀 나머지 부분 추출
face_crown = cv2.add(roi_bg, c_fg)


cv2.imshow('mask', mask)

cv2.imshow('mask_inv', mask_inv)
cv2.waitKey()
cv2.destroyAllWindows()

NameError: name 'roi' is not defined

In [39]:
glasses_img = cv2.imread('img/glasses.png')
ggray = cv2.cvtColor(glasses_img, cv2.COLOR_BGR2GRAY)
ggray[ggray[:]==0]=255   
ggray[ggray[:]!=255]=0

ggray_inv = cv2.bitwise_not(ggray)

cv2.imshow('gmask', ggray)

cv2.imshow('gmask_inv', gmask_inv)
cv2.waitKey()
cv2.destroyAllWindows()